In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.preprocessing as sp
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

%matplotlib inline

In [25]:
df_train1 = pd.read_csv('train1.csv',parse_dates=['date'])
# df_test=pd.read_csv("test.csv",parse_dates=['date'])

In [26]:
#for 2017,using average speed to fill the missing data
df_range = pd.date_range("2017-01-01 00:00:00","2017-12-31 23:00:00",freq="1H")
df_2017 = pd.DataFrame(df_range,columns=["date"])
df_train = df_train1.merge(df_2017,on="date",how="outer")
df_train = df_train.fillna(value={"speed":df_train["speed"].mean()})

In [27]:
# complete time series
df_range2 = pd.date_range("2017-01-01 00:00:00","2018-12-31 23:00:00",freq="1H")
new_df2=pd.DataFrame(df_range2,columns=["date"])
df_all=df_train.merge(new_df2,on="date",how="outer")

In [28]:
#extract year,month,day,hour
def extract_date(df,column):
    df[column+'_year'] = df[column].apply(lambda x:x.year)
    df[column+'_month'] = df[column].apply(lambda x:x.month)
    df[column+'_day'] = df[column].apply(lambda x:x.day)
    df[column+'_hour'] = df[column].apply(lambda x:x.hour)
    df[column+'_week'] = df[column].apply(lambda x:x.isoweekday())#1 represents MON,7 represents SUN
    df[column+'_date'] = df[column].apply(lambda x:x.strftime('%Y-%m-%d'))
extract_date(df_all,'date')

In [29]:
df_all['date_date'] = pd.to_datetime(df_all['date_date'])

In [30]:
#holiday label
df_all.loc[df_all['date_date'].isin(
    ['2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14','2017-04-15','2017-04-17',
     '2017-05-01','2017-05-03','2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26',
     '2018-01-01','2018-02-16','2018-02-17','2018-02-19','2018-03-30','2018-03-31','2018-04-02','2018-04-05',
     '2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17','2018-12-25','2018-12-26']),'vacation']=1

df_all.loc[~df_all['date_date'].isin(
    ['2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14','2017-04-15','2017-04-17',
     '2017-05-01','2017-05-03','2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26',
     '2018-01-01','2018-02-16','2018-02-17','2018-02-19','2018-03-30','2018-03-31','2018-04-02','2018-04-05',
     '2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17','2018-12-25','2018-12-26']),'vacation']=0

In [31]:
#weekend label
df_all.loc[df_all['date_week'].isin([1, 2, 3,4, 5]), 'week_en'] = 1
df_all.loc[df_all['date_week'].isin([6, 7]), 'week_en'] = 2

# time bin
df_all.loc[df_all['date_hour'].isin([7, 8,9]), 'hour_en'] = 1
df_all.loc[df_all['date_hour'].isin([17, 18,19]), 'hour_en'] = 2


In [32]:
df_all['day_label'] = df_all['date_year'].astype('str')+df_all['date_month'].astype('str')+ df_all['date_day'].astype('str')

In [33]:
df_all.head()

,id,date,speed,date_year,date_month,date_day,date_hour,date_week,date_date,vacation,week_en,hour_en,season,day_label
0,0.0,2017-01-01 00:00:00,43.002930,2017,1,1,0,7,2017-01-01,0.0,3.0,3.0,1.0,201711
1,1.0,2017-01-01 01:00:00,46.118696,2017,1,1,1,7,2017-01-01,0.0,3.0,3.0,1.0,201711
2,2.0,2017-01-01 02:00:00,44.294158,2017,1,1,2,7,2017-01-01,0.0,3.0,3.0,1.0,201711
3,3.0,2017-01-01 03:00:00,41.067468,2017,1,1,3,7,2017-01-01,0.0,3.0,3.0,1.0,201711
4,4.0,2017-01-01 04:00:00,46.448653,2017,1,1,4,7,2017-01-01,0.0,3.0,3.0,1.0,201711


In [34]:
params = {
        'learning_rate': 0.1,
        'n_estimators': 2500,
        'subsample': 0.9,
        'colsample_bytree': 0.9,
        'max_depth': 10,
        'min_child_weight': 1,
        'reg_alpha': 2,
        'gamma': 2
    }

In [35]:
df_all = pd.get_dummies(df_all, columns=['week_en','hour_en'])
# df_all = pd.get_dummies(df_all, columns=['date_month', 'week_en','date_year','hour_en','date_hour','worse_rainy'])
df_all = pd.get_dummies(df_all, columns=['day_label'])
# df_all = pd.get_dummies(df_all, columns=['date_hour'])

df_all.head()

,id,date,speed,date_year,date_month,date_day,date_hour,date_week,date_date,vacation,...,day_label_2018928,day_label_2018929,day_label_201893,day_label_2018930,day_label_201894,day_label_201895,day_label_201896,day_label_201897,day_label_201898,day_label_201899
0,0.0,2017-01-01 00:00:00,43.002930,2017,1,1,0,7,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2017-01-01 01:00:00,46.118696,2017,1,1,1,7,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,2017-01-01 02:00:00,44.294158,2017,1,1,2,7,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,2017-01-01 03:00:00,41.067468,2017,1,1,3,7,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4.0,2017-01-01 04:00:00,46.448653,2017,1,1,4,7,2017-01-01,0.0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
feature = df_all.columns.values.tolist()
train_feature = [x for x in feature if
                 x not in ['id', 'date','date_date','speed']]

train_df = df_all.loc[~df_all['speed'].isnull()]
test_df = df_all.loc[df_all['speed'].isnull()].copy()

print (train_feature)

['date_year', 'date_month', 'date_day', 'date_hour', 'date_week', 'vacation', 'season', 'week_en_1.0', 'week_en_3.0', 'hour_en_1.0', 'hour_en_2.0', 'hour_en_3.0', 'hour_en_4.0', 'hour_en_5.0', 'hour_en_6.0', 'day_label_2017101', 'day_label_20171010', 'day_label_20171011', 'day_label_20171012', 'day_label_20171013', 'day_label_20171014', 'day_label_20171015', 'day_label_20171016', 'day_label_20171017', 'day_label_20171018', 'day_label_20171019', 'day_label_2017102', 'day_label_20171020', 'day_label_20171021', 'day_label_20171022', 'day_label_20171023', 'day_label_20171024', 'day_label_20171025', 'day_label_20171026', 'day_label_20171027', 'day_label_20171028', 'day_label_20171029', 'day_label_2017103', 'day_label_20171030', 'day_label_20171031', 'day_label_2017104', 'day_label_2017105', 'day_label_2017106', 'day_label_2017107', 'day_label_2017108', 'day_label_2017109', 'day_label_201711', 'day_label_2017110', 'day_label_2017111', 'day_label_20171110', 'day_label_20171111', 'day_label_20

In [37]:
X = train_df[train_feature].values
y = train_df['speed'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

eval_set = [(X_test, y_test)]


In [38]:
regressor = xgb.XGBRegressor(learning_rate=params['learning_rate'], n_estimators=params['n_estimators'],
                             booster='gbtree', objective='reg:linear', n_jobs=-1, subsample=params['subsample'],
                             colsample_bytree=params['colsample_bytree'], random_state=0,
                             max_depth=params['max_depth'], gamma=params['gamma'],
                             min_child_weight=params['min_child_weight'], reg_alpha=params['reg_alpha'])

# regressor.fit(X_train, y_train, verbose=True, early_stopping_rounds=10,eval_set=eval_set)
regressor.fit(X_train, y_train, verbose=True,eval_set=eval_set)
predict_ytrain=regressor.predict(X_train)
predict_ytest=regressor.predict(X_test)
error1 = mean_squared_error(y_train,predict_ytrain)
error2 = mean_squared_error(y_test,predict_ytest)
# regressor.fit(X, y, verbose=True, eval_set=eval_set)
# predict_y=regressor.predict(X)
# error = mean_squared_error(y,predict_y)
print(error1)
print(error2)
test_df['prediction'] = regressor.predict(test_df[train_feature].values)

[23:01:26] WARNING: /Users/gaowanqi/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[2499]	validation_0-rmse:3.58396
1.9595544850736772
12.844750124280852
       date_year  date_month  date_day  date_hour  date_week  vacation  \
14016       2018           1         1          2          1       1.0   
14017       2018           1         1          5          1       1.0   
14018       2018           1         1          7          1       1.0   
14019       2018           1         1          8          1       1.0   
14020       2018           1         1         10          1       1.0   

       season  week_en_1.0  week_en_3.0  hour_en_1.0  ...  day_label_2018928  \
14016     1.0            1            0            0  ...                  0   
14017     1.0            1            0            0  ...                  0   
14018     1.0            1            0            0  ...                  0   
14019     1.0           

In [18]:
df_test=pd.read_csv("test.csv",parse_dates=['date'])

In [19]:
df_result=pd.merge(df_test,test_df[['prediction','date']],on='date',how='left')

In [20]:
df_result=df_result.drop(['date','id'],axis=1)

In [21]:
df_result.to_csv('df_result.csv', header=True,index=True,sep=';', mode='w')

In [22]:
df_result

,prediction
0,48.447571
1,48.068596
2,37.840508
3,26.204849
4,38.809216
...,...
3499,13.312586
3500,26.234112
3501,47.684368
3502,41.867733
